crewai 捨棄要錢的 openAI，改使用免費的 Ollama 或 gemini pro

Ollama 現階段還是不夠強，勉強可以用 openhermes，否則就只能用 gemini pro

crewai 必須在建立 agent、task 時就把角色、任務設定好，才能執行，

phidata 則可以透過 cli 交互命令行來交付任務，比較有彈性。

autogen 則是預設就有提供執行 python 程式碼的功能，可以自動產生程式碼。


In [ ]:
!pip install --q crewai
!pip install --q -U duckduckgo-search
!pip install --q langchain_google_genai

In [1]:
# 匯入依賴庫
import os
from crewai import Agent, Task, Crew, Process


In [2]:
import textwrap
from IPython.display import HTML, display, Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
# 設定4個 LLM
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.llms import OpenAI, Ollama
from langchain_openai import ChatOpenAI
# crewai 使用 Ollama 的方法：ollama 已推出 windows 版本，不需要執行 litellm；Mac 則也不需要啟動任何程式，只要有安裝好 Ollama 即可調用。
# (autogen 要調用 Ollama 需要執行 litellm，多一道步驟，比較麻煩)
gpt35 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7),
gpt4 = ChatOpenAI(model_name="gpt-4", temperature=0.7),
# Ollama 大模型，用 modelfile 建立的，可以在 modelfile 設定溫度，但切勿在系統提示詞要它使用中文回復，會造成函數調用不正常
ollama = Ollama(model="openhermes_assistant")  # 這是用 modelfile 建立的
ollama_openhermes = Ollama(model="openhermes")

gemini = ChatGoogleGenerativeAI(model="gemini-pro",
                             verbose = True,
                             temperature = 0.6,
                             google_api_key = os.environ["GOOGLE_API_KEY"]
                             )

In [4]:
# 設定工具 tools
# Make sure to Install duckduckgo-search for this example:
# !pip install -U duckduckgo-search
# 最簡易型的工具定義方式
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

# 從 tools 模組中引入需要的工具
from tools.browser_tools import BrowserTools
from tools.calculator_tools import CalculatorTools
from tools.file_tools import FileTools
from tools.search_tools import SearchTools
from tools.sec_tools import SECTools
# 下面添加 tools 時，必須用 類別.方法 加入工具
# BrowserTools.scrape_and_summarize_website
# CalculatorTools.calculate
# FileTools.write_file
# SearchTools.search_internet、SearchTools.search_news
# SECTools.search_10q、SECTools.search_10k
# 下面的是依樣畫葫蘆自行加入2行定義類別，親測可以匯入
from tools.find_papers_arxiv import FindPapersArxiv
from tools.get_top_hackernews_stories import GetTopHackernewsStories
# 然後就可以用下面的函式來使用工具，已測試可以~~~
# FindPapersArxiv.search_arxiv
# GetTopHackernewsStories.get_top_hackernews_stories

In [5]:
# Define your agents with roles and goals
# allow_delegation=True, 打開測試看看
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting
  actionable insights.""",
  verbose=True,
  allow_delegation=True,
  tools=[
        search_tool, 
        BrowserTools.scrape_and_summarize_website, 
        CalculatorTools.calculate, 
        FileTools.write_file,
        SearchTools.search_internet, SearchTools.search_news,
        SECTools.search_10q, SECTools.search_10k,
        FindPapersArxiv.search_arxiv,
        GetTopHackernewsStories.get_top_hackernews_stories
      ],
  # You can pass an optional llm attribute specifying what mode you wanna use.
  # It can be a local model through Ollama / LM Studio or a remote
  # model like OpenAI, Mistral, Antrophic of others (https://python.langchain.com/docs/integrations/llms/)
  #
  # Examples:
  # llm=ollama # was defined above in the file
  # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7)
  # llm = gemini  #using google gemini pro API
  # 或在定義後，使用 researcher.llm = ollama 或 gemini 來更改模型
)

In [6]:
# 寫手應該不需要掛太多 tools，單純測試看看。
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for
  your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True,
  tools=[
        search_tool, 
        BrowserTools.scrape_and_summarize_website, 
        CalculatorTools.calculate, 
        FileTools.write_file,
        SearchTools.search_internet, SearchTools.search_news,
        SECTools.search_10q, SECTools.search_10k,
        FindPapersArxiv.search_arxiv,
        GetTopHackernewsStories.get_top_hackernews_stories
      ],
  # llm = gemini  #using google gemini pro API
)


In [7]:
# 上面可以先不指定用哪一個 llm，這裡再指定就可以
# 但是如果分 main.py、agents.py、tasks.py 三個檔案，就必須先指定用哪一個 llm
researcher.llm = ollama
writer.llm = ollama

In [8]:
# Create tasks for your agents
# 跟 phidata 比較，phidata 有提供 cli 命令交互式的方法，透過交互式的方式，可以讓使用者在命令列輸入文字，並回傳答案。
# 但 crewai 必須在程式碼中定義好要執行的任務 tasks，並執行任務。
# 要 AI 解決任務，就必須透過程式碼內輸入 tasks 的任務內容，並執行程式，才能得到結果。
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report""",
  agent=researcher
)
task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.
  Your final answer MUST be the full blog post of at least 5 paragraphs.""",
  agent=writer
)
task3 = Task(
  description="""express the final blog post provided in traditional chinese。 and write it to a file AI_2024.md""",
  agent=writer
)

In [9]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2, task3],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

In [11]:
# crew
# Get your crew to work!
result = crew.kickoff()
# print(result)
to_markdown(result)


[DEBUG]: Working Agent: Senior Research Analyst
[INFO]: Starting Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report


> Entering new CrewAgentExecutor chain...
Use Tool: duckduckgo_search
Query: "latest advancements in AI 2024" 

In 2024, generative AI might actually become useful for the regular, non-tech person, and we are going to see more people tinkering with a million little AI models. State-of-the-art AI... 10 top AI and machine learning trends for 2024 Custom enterprise models, open source AI, multimodal -- learn about the top AI and machine learning trends for 2024 and how they promise to transform the industry. By Lev Craig, Site Editor Published: 04 Jan 2024 published 27 December 2023 While 2023 was the year of AI, 2024 will be the year we use it. Comments (0) (Image credit: Aston Martin/Apple) We have just come to

> 此博客文章的標題為：《AI的未來：2024年最重要的十大進步》。
> 
> 在接近未來，人工智能技術將重塑行業並改變我們日常生活。今天我們將更瞭解2024年對十個關鍵趨勢和科技的影響。從量子AI到可解釋AI，這些進步將驅動跨行業创新，包括健康、金融、交通和製造等行業。
> 
> 1. 量子AI：量子計算即將成為提升人工智能功能的關鍵因素。在2024年，我們將見到這個初創性領域中的重大進展，量子AI將歧視金融、健康和數字安全等行業，使得資料處理和分析更快速。
> 2. 可解釋AI：随著人工智能在每個角落深入人心，有需要透明、可責任的人工智能系統成為相關的趨勢。可解釋AI將使得人工智能模型的決策過程更容易理解和追溯，確保其輸出可以被驗證和檢驗，從而提升信任於人工智能驅動的解決方案。
> 3. 多模式AI：在2024年，AI將變得更為多元化，因為它可以同時處理和理解不同類型的數據（如文本、圖像、聲音和視頻）。這種能力將使得AI模型可以理解和回應複雜的人類查詢和互動，最終提高使用者體驗。
> 4. 企業定制化AI：随著人工智能成為主流，企業將投資以其特定需求為目標的自定義AI。這些精巧解決問題的礬銷類型AI在苛家、客戶服務、產品研究等區域的礽擒，利用人工智能的特殊幾項力將。
> 5. 開源AI：開源運動將在導進銷關鍵的幾關釋人工智能技術。通過使人工AI輪件和資源免費，更多的個體和組織可以贡獻於人工智能的進展。
> 6. 邊條AI：與互联網設備裝置和需要即時決策，邊條AI將成為越重要的類型AI。這種AI在本地處理數據而不是將其發送到中央伺服器，降低延誤並改進表現。
> 7. 增強實際實境（AR）和虛實際實區（VR）：人工智能與AR和VR技術的整合將導致在這些領域的重大進展。AI加強度的補援和虛實實區應用程式將使得更個體化、個體化的經捗疾教育、娛教育和市場。
> 8. 人工AI合作：随著人工智能演變，將會有關注建立與人工AI一起工作的平鋒質態系統。這涉及在AI系統與人工協力力量補援，舍辯其強烈質性。
> 9. 合成資料生成：能力生成合成數據將成為開發和測試人工智能模型的重要因素。這將使得研究員和開發者可以創造更多具有各種表現和性質的數據集，降低過合釋風潤並改進模型的一般化。
> 10. 信任性AI：随著人工智能對各個方面的生活產生影響，有需要責任、責任和責報答的可解釋和人工智能系統。這涉及在確保AI模型的透明性、公平性和責報應地，面對資料隱密和安全性相關的問題。
> 
> 這些人工智能進展將具有重大影響於各個行業，包括健康、金融、交通和製造等。然而，我們需要面對由於這些嚴快進展的挑戰，如責設立合銷的開發、保鍵眾私人資料隱密和賽報應學力工作力量。
> 
> 請記住，您可以<|im_start|>與我們聯繫，如果您需要任何幫助或有進一步問題！